In [ ]:
# https://machinelearningmastery.com/moving-average-smoothing-for-time-series-forecasting-python/

In [ ]:
import chardet
with open('james_training.xlsx', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

In [ ]:
from pandas import read_csv
from matplotlib import pyplot
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

series = read_csv(r'new_james.txt', header=0, index_col=0,sep='\t')


In [ ]:
series.head()

In [ ]:
series.shape

In [ ]:
series.boxplot(grid='false', color='blue',fontsize=10, rot=30, showfliers=False )

In [ ]:
df=series

In [ ]:
Q1_return = df['Returns'].quantile(0.25)
Q3_return = df['Returns'].quantile(0.75)
IQR_return = Q3_return - Q1_return    #IQR is interquartile range. 

filter = (df['Returns'] >= Q1_return - 1.5 * IQR_return) & (df['Returns'] <= Q3_return + 1.5 *IQR_return)
df.loc[filter]

In [ ]:
Q1_return

In [ ]:
Q3_return


In [ ]:
Q1 = df['Sales'].quantile(0.25)
Q3 = df['Sales'].quantile(0.75)
IQR = Q3 - Q1    #IQR is interquartile range. 

filter = (df['Sales'] >= Q1 - 1.5 * IQR) & (df['Sales'] <= Q3 + 1.5 *IQR)
df.loc[filter]

In [ ]:
Q3

In [ ]:
Q1

In [ ]:

plt.rcParams["figure.figsize"] = (20,3)
print(series.head())
figure(figsize=(18, 6), dpi=80)
series.plot()
pyplot.show()

In [ ]:
series.boxplot(column=['Sales'])

In [ ]:
print(Q1)
print(Q3)

In [ ]:
df2=df

In [ ]:
df=df2

In [ ]:
df = df[df['Sales'] < Q3]
df= df[df['Sales'] > Q1]

In [ ]:
df.shape

In [ ]:
df = df[df['Returns'] < Q3_return]
df = df[df['Returns'] > Q1_return]

In [ ]:
df.shape

In [ ]:

plt.rcParams["figure.figsize"] = (20,3)
print(series.head())
figure(figsize=(18, 6), dpi=80)
df.plot()
pyplot.show()

In [ ]:
# Tail-rolling average transform
rolling = df.rolling(window=3)
rolling_mean = rolling.mean()
print(rolling_mean.head(10))
# plot original and transformed dataset
series.plot()
rolling_mean.plot(color='red')
pyplot.show()

In [ ]:
## creating varibales
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

df = DataFrame(df.values)


In [ ]:
df.head()

In [ ]:
df_Sales= df[0]
df_Return=df[1]

In [ ]:
lag2= df_Sales.shift(2)
lag2.head()

In [ ]:
width = 3
lag1 = df_Sales.shift(1)
lag2= df_Sales.shift(2)
#lag
lag3 = df_Sales.shift(width - 1)

In [ ]:
window = lag3.rolling(window=width)

In [ ]:
means = window.mean()

In [ ]:
dataframe = concat([means, lag1,lag2, df_Sales], axis=1)

In [ ]:
dataframe

In [ ]:
dataframe.columns = ['mean', 't-1', 't+1','t+2']

In [ ]:
dataframe.head()

In [ ]:
df = DataFrame(dataframe.values)


In [ ]:
#df = DataFrame(series.values)
width = 3
lag1 = df.shift(1)
lag2= df.shift(2)
lag3 = df.shift(width - 1)
window = lag3.rolling(window=width)
means = window.mean()
dataframe = concat([means, lag1,lag2, df], axis=1)
dataframe.columns = ['mean', 't-1', 't+1','t+2']
print(dataframe.head(10))

In [ ]:

from pandas import read_csv
from matplotlib import pyplot
series =  read_csv(r'new_james.txt', header=0, index_col=0,sep='\t')
# Tail-rolling average transform
rolling = series.rolling(window=3)
rolling_mean = rolling.mean()
print(rolling_mean.head(10))
# plot original and transformed dataset
series.plot()
rolling_mean.plot(color='red')
pyplot.show()

## Moving Average as Feature Engineering

In [1]:
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

import pandas as pd
#series = read_csv('daily-total-female-births-CA.csv', header=0, index_col=0)
df= pd.read_excel('james_training.xlsx')

In [2]:
series=df['Sales']

In [3]:
series.head()

0    5549
1    4285
2    3950
3    3996
4    4413
Name: Sales, dtype: int64

df = DataFrame(series.values)
width = 3
lag1 = df.shift(1)
lag2= df.shift(2)
lag3 = df.shift(width - 1)
window = lag3.rolling(window=width)
means = window.mean()
dataframe = concat([means, lag1,lag2, df], axis=1)
dataframe.columns = ['mean', 't-1', 't+1','t+2']
print(dataframe.head(10))

In [5]:
######  https://machinelearningmastery.com/random-forest-for-time-series-forecasting/

In [6]:
values = series.values

In [9]:
values.shape

(366,)

In [ ]:
## Random forest for time series prediction

In [10]:
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot
 
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values
 
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test, :], data[-n_test:, :]
 
# fit an random forest model and make a one step prediction
def random_forest_forecast(train, testX):
    # transform list into array
    train = asarray(train)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
    # fit model
    model = RandomForestRegressor(n_estimators=10000)
    model.fit(trainX, trainy)
    # make a one-step prediction
    yhat = model.predict([testX])
    return yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # split test row into input and output columns
        testX, testy = test[i, :-1], test[i, -1]
        # fit model on history and make a prediction
        yhat = random_forest_forecast(history, testX)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
        # summarize progress
        print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
    # estimate prediction error
    error = mean_absolute_error(test[:, -1], predictions)
    return error, test[:, -1], predictions


In [11]:
values=series.values

In [12]:
len(values)

366

 
# load the dataset
series = read_csv('daily-total-female-births-CA.csv', header=0, index_col=0)
values = series.values


In [14]:
# transform the time series data into supervised learning
data = series_to_supervised(values, n_in=3)

IndexError: tuple index out of range

In [ ]:
data

In [ ]:
# evaluate
mae, y, yhat = walk_forward_validation(data, 12)
print('MAE: %.3f' % mae)
# plot expected vs predicted
pyplot.plot(y, label='Expected')
pyplot.plot(yhat, label='Predicted')
pyplot.legend()
pyplot.show()